In [1]:
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_bloch_multivector
from math import sqrt, pi
from numpy.random import randint
import numpy as np

from qiskit import IBMQ
from qiskit_ibm_provider import IBMProvider, least_busy
apitoken = 'add822bdd7a0cf338059a6157b26e50155c5bba4fc90826592b20ff80f379a99c8aefe0e52ad3dd304ead3d46edca9af9a9c507bd0096c1c085ce7b369c8e422' #insert api token here

# #for account loading
# IBMProvider.save_account(token=apitoken)
# IBMProvider.load_account()

# Setting the random seed
np.random.seed(seed=60) # You may choose any seed instead of just `0'
n = 4 # This is the number of bits in the string
alice_bits = randint(2, size=n)
print(alice_bits)

[1 1 0 0]


In [2]:
provider = IBMProvider()
backend = least_busy(provider.backends(filters=lambda x:
x.configuration().n_qubits >= 2
and not x.configuration().simulator
and x.status().operational==True))
print("least busy backend: ", backend)

least busy backend:  <IBMBackend('ibm_lagos')>


In [3]:
q = QuantumRegister(4)
c = ClassicalRegister(4)

In [4]:
circuit = QuantumCircuit(q,c)
circuit.h(q[0])
circuit.h(q[2])
circuit.cx(q[0], q[1])
circuit.cx(q[2], q[3])
if alice_bits[0] == 0:
    if alice_bits[1] == 1:
        circuit.x(q[0])
elif alice_bits[0] == 1:
    if alice_bits[1] == 0:
        circuit.z(q[0])
    else:
        circuit.ry(pi, q[0])
        
if alice_bits[2] == 0:
    if alice_bits[3] == 1:
        circuit.x(q[2])
elif alice_bits[2] == 1:
    if alice_bits[3] == 0:
        circuit.z(q[2])
    else:
        circuit.ry(pi, q[2])
        
circuit.i(q[0])
circuit.i(q[2])
        
circuit.cx(q[0], q[1])
circuit.cx(q[2], q[3])
circuit.h(q[0])
circuit.h(q[2])
circuit.measure(q[0],c[3])
circuit.measure(q[1],c[2])
circuit.measure(q[2],c[1])
circuit.measure(q[3],c[0])
circuit.draw()

┌───┐     ┌───────┐┌───┐     ┌───┐   ┌─┐
q0_0: ┤ H ├──■──┤ Ry(π) ├┤ I ├──■──┤ H ├───┤M├
      └───┘┌─┴─┐└───────┘└───┘┌─┴─┐└┬─┬┘   └╥┘
q0_1: ─────┤ X ├──────────────┤ X ├─┤M├─────╫─
      ┌───┐└───┘  ┌───┐       ├───┤ └╥┘ ┌─┐ ║ 
q0_2: ┤ H ├──■────┤ I ├────■──┤ H ├──╫──┤M├─╫─
      └───┘┌─┴─┐  └───┘  ┌─┴─┐└┬─┬┘  ║  └╥┘ ║ 
q0_3: ─────┤ X ├─────────┤ X ├─┤M├───╫───╫──╫─
           └───┘         └───┘ └╥┘   ║   ║  ║ 
c0: 4/══════════════════════════╩════╩═══╩══╩═
                                0    2   1  3

In [5]:
simulator = Aer.get_backend('statevector_simulator')
job = execute(circuit, simulator)
result = job.result()
statevector = result.get_statevector()
print(statevector)

Statevector([ 0.+0.000000e+00j,  0.+0.000000e+00j,  0.-0.000000e+00j,
             -1.+6.123234e-17j,  0.+0.000000e+00j,  0.+0.000000e+00j,
              0.+0.000000e+00j,  0.+0.000000e+00j,  0.+0.000000e+00j,
              0.+0.000000e+00j,  0.+0.000000e+00j,  0.+0.000000e+00j,
              0.+0.000000e+00j,  0.+0.000000e+00j,  0.+0.000000e+00j,
              0.+0.000000e+00j],
            dims=(2, 2, 2, 2))


In [6]:
# simulator = Aer.get_backend('qasm_simulator')
# job = execute(circuit, simulator, shots = 1024)
# result = job.result()
# counts = result.get_counts()
# plot_histogram(counts)

qcomp = provider.get_backend("ibm_lagos")
job = execute(circuit, backend = qcomp, shots = 1000)
from qiskit.tools.monitor import job_monitor
job_monitor(job)

Job Status: job has successfully run
